<h1>Table of Contens<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Scrapper-Post-Offices" data-toc-modified-id="Scrapper-Post-Offices-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Scrapper Post Offices</a></span></li><li><span><a href="#Libraries" data-toc-modified-id="Libraries-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Libraries</a></span></li><li><span><a href="#Page-Source" data-toc-modified-id="Page-Source-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Page Source</a></span></li><li><span><a href="#Browser-Firefox" data-toc-modified-id="Browser-Firefox-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Browser Firefox</a></span></li><li><span><a href="#Process-page" data-toc-modified-id="Process-page-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Process page</a></span></li><li><span><a href="#Convert-Result-List-to-Dataframe" data-toc-modified-id="Convert-Result-List-to-Dataframe-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Convert Result List to Dataframe</a></span></li><li><span><a href="#New-column---post-office-agency-type" data-toc-modified-id="New-column---post-office-agency-type-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>New column - post office agency type</a></span></li><li><span><a href="#Summarization-table" data-toc-modified-id="Summarization-table-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Summarization table</a></span></li><li><span><a href="#Detailed-summarization-table" data-toc-modified-id="Detailed-summarization-table-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Detailed summarization table</a></span></li><li><span><a href="#SAVE" data-toc-modified-id="SAVE-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>SAVE</a></span></li><li><span><a href="#Close-Browser-;-)" data-toc-modified-id="Close-Browser-;-)-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Close Browser ;-)</a></span></li></ul></div>

# Scrapper Post Offices

Scrapper to count Post Offices per City, In Brazil

# Libraries

In [ ]:
# To instal Selenium: conda install -c conda-forge selenium 
from selenium import webdriver 
from selenium.webdriver.support.ui import Select

# To install BeautifulSoup: conda install -c anaconda beautifulsoup4
from bs4 import BeautifulSoup

# Donwload geckodriver: https://github.com/mozilla/geckodriver

import pandas as pd
 
import time

In [ ]:
pd.options.display.max_colwidth = 150
pd.options.display.max_rows = 500

# Page Source

In [ ]:
src = "http://www2.correios.com.br/sistemas/agencias/"

# Browser Firefox

In [ ]:
browser = webdriver.Firefox()  
browser.get(src)  

# Process page

In [ ]:
result = []

In [ ]:
# select the state drop down list
select = Select(browser.find_element_by_id("estadoAgencia"))
states= [ option.text for option in select.options ] 
print(states)

In [ ]:
# select the state drop down list
select = Select(browser.find_element_by_id("estadoAgencia"))

for option in (select.options):
        
        if option.text : # if this is not an empty value
        
            print(option.text)
            #select.select_by_visible_text("SÃO PAULO")
            select.select_by_visible_text(option.text)

            # select the city
            select2 = Select(browser.find_element_by_id("municipioAgencia"))

            for option2 in select2.options:
                
                if option2.text: # if this is not an empty value

                    select2.select_by_visible_text(option2.text)
                    #select2.select_by_visible_text("AGUAS DA PRATA")

                    # Sleep... otherwise you will get blocked
                    time.sleep(25)

                    #soup = BeautifulSoup(html, 'lxml')
                    #a = soup.find('section', 'wrapper')
                    html_source = browser.page_source 
                    soup = BeautifulSoup(html_source,'html.parser')

                    results = soup.find('div',{'class':'resultadoAgencia'})

                    # Look for: <table class="agencia listEvent" id="tableNomeAgencia">

                    if results.findAll('table', id="tableNomeAgencia"):

                        agencies = results.findAll('table', id="tableNomeAgencia")      

                        '''
                        <table class="agencia listEvent" id="tableNomeAgencia">
                            <tbody><tr>
                            <td class="bullet ageNomeEvent nomeAgencia"><a href="javaScript:mostrarDetalheAgencia('2','1','','')">AC AGUAS DA PRATA              - AGENCIA CORREIO               </a></td>
                            <td class="text-right">
                            </td>
                            </tr>
                            </tbody></table>
                        '''
                        for agency in agencies:
                            name= agency.find("td")
                            # print(option.text, option2.text, name.text)
                            result = result + [[option.text, option2.text, " ".join(name.text.split())]]
        
       

# Convert Result List to Dataframe

In [ ]:
df = pd.DataFrame(result, columns = ["STATE","CITY","AGENCY"])

In [ ]:
df.shape

In [ ]:
df.tail()

# New column - post office agency type

In [ ]:
df["TYPE"] = df["AGENCY"].str.partition("-")[2].str.strip()

In [ ]:
df["TYPE"].value_counts()

In [ ]:
mask = df["TYPE"] == "LANDIA - AG CORREIOS COMUNITARIA"
df[mask]

In [ ]:
df.loc[mask,"AGENCY"] = "AGC LEALANDIA - AG CORREIOS COMUNITARIA"

In [ ]:
df["TYPE"] = df["AGENCY"].str.partition("-")[2].str.strip()

In [ ]:
df["TYPE"].value_counts()

In [ ]:
df["TYPE"].str.partition("(")[0].str.strip().value_counts()

In [ ]:
df["TYPE"] = df["TYPE"].str.partition("(")[0].str.strip()

In [ ]:
df["TYPE"].value_counts()

# Summarization table

In [ ]:
df.pivot_table(index=["STATE","CITY"], values="AGENCY",aggfunc="count").reset_index().sort_values(by="AGENCY", ascending=False).head(20)

In [ ]:
table1= df.pivot_table(index=["STATE","CITY"], values="AGENCY",aggfunc="count").reset_index().sort_values(by="AGENCY", ascending=False)

# Detailed summarization table

In [ ]:
df.pivot_table(index=["STATE","CITY"], columns="TYPE", values="AGENCY",aggfunc="count", fill_value=0).reset_index().sort_values(by="AG CORREIOS COMUNITARIA", ascending=False).head(20)

In [ ]:
table2 = df.pivot_table(index=["STATE","CITY"], columns="TYPE", values="AGENCY",aggfunc="count", fill_value=0).reset_index().sort_values(by="AG CORREIOS COMUNITARIA", ascending=False)

# SAVE

In [ ]:
writer = pd.ExcelWriter( "data_final.xlsx" )
        
df.to_excel(writer, sheet_name= "DATA", index=False)
table1.to_excel(writer, sheet_name= "TABLE1", index=False)
table2.to_excel(writer, sheet_name= "TABLE2", index=False)
        
writer.save()    
        

# Close Browser ;-)

In [ ]:
browser.close()